In [1]:
using Statistics
using TextAnalysis

# ファイルを読み込む
file_path = "./data/20231102_alice.txt"
text = read(file_path, String)

# テキストを小文字に変換
text = lowercase(text)
doc = StringDocument(text)
# prepare!(doc, strip_punctuation)
# テキストを文に分割する

A StringDocument{String}
 * Language: Languages.English()
 * Title: Untitled Document
 * Author: Unknown Author
 * Timestamp: Unknown Time
 * Snippet: alice was beginning to get very tired of sitting b

In [2]:
include("./lib/expand_contractions.jl")
text = expand_contractions(text)

"alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought alice “wi" ⋯ 148718 bytes ⋯ "yes bright and eager with many a strange tale, perhaps even with the dream of wonderland of long ago: and how she would feel with all their simple sorrows, and find a pleasure in all their simple joys, remembering her own child-life, and the happy summer days.\n\nthe end"

In [3]:
sentences = split(text, r"[.?!“”(),\n]+")

4861-element Vector{SubString{String}}:
 "alice was beginning to get very tired of sitting by her sister on the bank"
 " and of having nothing to do: o" ⋯ 33 bytes ⋯ "the book her sister was reading"
 " but it had no pictures or conversations in it"
 " "
 "and what is the use of a book"
 " thought alice "
 "without pictures or conversations"
 "so she was considering in her own mind "
 "as well as she could"
 " for the hot day made her feel very sleepy and stupid"
 ⋮
 " be herself a grown woman; and how she would keep"
 " through all her riper years"
 " the simple and loving heart of" ⋯ 41 bytes ⋯ "about her other little children"
 " and make their eyes bright and eager with many a strange tale"
 " perhaps even with the dream of" ⋯ 46 bytes ⋯ "l with all their simple sorrows"
 " and find a pleasure in all their simple joys"
 " remembering her own child-life"
 " and the happy summer days"
 "the end"

In [4]:

# 各文を単語に分割し、STARTとENDを追加する
processed_sentences :: Vector{Vector{String}} = []
for sentence in sentences
    words :: Vector{String} = split(sentence)
    if isempty(words) || length(words) < 3
        continue
    end
    pushfirst!(words, "START")
    push!(words, "END")
    push!(processed_sentences, words :: Vector{String})
end

In [9]:
include("./lib/probability.jl")

calculator = TextEntropyCalculator(processed_sentences)
entropy = calculate_entropy(calculator, "START is the rabbit on the table white END")
println("Cross Entropy: ", entropy)

Calculating bigrams and singlegrams...total_bigrams: 28789
total_singlegrams: 32252
0.05440851049214694
0.0077421456764336614
0.07030571747488529
0.0006666253255612055
0.39817036765702646
0.0468291939983618
0.0002635495473148952
0.029861349227772466
0.05368659307949895
Cross Entropy: 3.639967087964231


In [6]:
p_sentences = []

for sentence in processed_sentences
    push!(p_sentences, join(sentence, " "))
end

# join sentences


processed_text = join(p_sentences, "\n")

output = "./data/alice_processed.txt"
open(output, "w") do f
    write(f, processed_text)
end

163047

In [12]:
include("./lib/calc_entropy.jl")
include("./lib/probability.jl")

# 指定された文
case1 = ["START is the rabbit on the table white END", "START does the rabbit on the table red END"]
case2 = ["START is the girl reading END", "START does the rabbit reading END"]
case3 = ["START is the caterpillar tired END", "START does the caterpillar tired END"]
case4 = ["START is the house old END", "does the house old END"]

function compare_2sentences(sentence1, sentence2)
    words1 = split(sentence1)
    words2 = split(sentence2)
    e1 = calculate_entropy(calculator, sentence1)
    e2 = calculate_entropy(calculator, sentence2)
    println("Cross Entropy: ", e1, " ", e2)
end
compare_2sentences(case1[1], case1[2])
compare_2sentences(case2[1], case2[2])
compare_2sentences(case3[1], case3[2])
compare_2sentences(case4[1], case4[2])

Cross Entropy: 3.639967087964231 3.7045342161114894
Cross Entropy: 3.737001591000174 3.67814255087666
Cross Entropy: 4.316340219154875 4.399234667949564
Cross Entropy: 3.381465347784939 3.5985978858774614


In [8]:
include("./lib/biagram_export.jl")
# バイグラムのデータセット
bigram_data = [
    ("START", "Alice", 5),
    ("Alice", "was", 15),
    ("was", "beginning", 10),
    ("beginning", "to", 8),
    ("to", "get", 12),
    ("get", "very", 7),
    ("very", "tired", 3),
    ("tired", "of", 4),
    ("of", "sitting", 2),
    ("sitting", "by", 6),
    ("by", "her", 9),
    ("her", "sister", 11),
    ("sister", "on", 13),
    ("on", "the", 14),
    ("the", "bank", 16),
    ("bank", "END", 1)
]

# 関数を呼び出してグラフを生成し、ファイルに保存
generate_bigram_graph(bigram_data, "sample_graph")

LoadError: LoadError: ArgumentError: Package Measures not found in current path.
- Run `import Pkg; Pkg.add("Measures")` to install the Measures package.
in expression starting at /workspaces/JuliaAliceBigram/lib/biagram_export.jl:2